In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
from prophet import Prophet
import pickle
import os
import psycopg2
from datetime import datetime


DB_HOST = "tu_host_postgres"
DB_NAME = "aemet"
DB_USER = "CarlosBaragano"
DB_PASS = "V!4Nbm$%w5b45$"
DB_PORT = 5432

def get_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )


app = FastAPI(title="AEMET Forecast API", version="1.0")


class AskRequest(BaseModel):
    question: str

class ForecastRequest(BaseModel):
    location: str
    days: int


MODEL_PATH = "model/prophet_model.pkl"
os.makedirs("model", exist_ok=True)

if os.path.exists(MODEL_PATH):
    with open(MODEL_PATH, "rb") as f:
        model = pickle.load(f)
    print("  Modelo cargado desde disco")
else:
    print("  Entrenando modelo Prophet con cadiz_final.json ...")
    df = pd.read_json("cadiz_final.json")
    df['fecha'] = pd.to_datetime(df['fecha'])
    df = df.sort_values('fecha')
    
   
    df_prophet = df[['fecha','tmed']].rename(columns={'fecha':'ds','tmed':'y'}).dropna()
    
    model = Prophet(daily_seasonality=True, yearly_seasonality=True)
    model.fit(df_prophet)
    
    # Guardar modelo
    with open(MODEL_PATH, "wb") as f:
        pickle.dump(model, f)
    print("  Modelo entrenado y guardado en model/prophet_model.pkl")


@app.post("/ask")
def ask(request: AskRequest):
    question = request.question.lower()
    
   
    temp_type = "tmax" if "máxima" in question else "tmin" if "mínima" in question else "tmed"
    location = "cadiz" if "cadiz" in question else None
    
    if not location:
        raise HTTPException(status_code=400, detail="No se reconoce la ubicación")
    
    try:
        conn = get_connection()
        query = f"""
            SELECT fecha, {temp_type} 
            FROM observaciones_cadiz
            ORDER BY fecha DESC
            LIMIT 7;
        """
        df = pd.read_sql(query, conn)
        conn.close()
        result = df.to_dict(orient="records")
        return {"location": location, "temperature_type": temp_type, "data": result}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.post("/forecast")
def forecast(request: ForecastRequest):
    location = request.location.lower()
    days = request.days
    
    if location != "cadiz":
        raise HTTPException(status_code=400, detail="Solo disponible para Cádiz actualmente")
    
    # Predecir próximos días
    future = model.make_future_dataframe(periods=days)
    forecast_df = model.predict(future)
    
    forecast_result = forecast_df[['ds','yhat','yhat_lower','yhat_upper']].tail(days)
    forecast_result['ds'] = forecast_result['ds'].dt.strftime("%Y-%m-%d")
    
    return {"location": location, "forecast": forecast_result.to_dict(orient="records")}


  Modelo cargado desde disco


In [2]:
import pickle
import pandas as pd


MODEL_PATH = "model/prophet_model.pkl"
with open(MODEL_PATH, "rb") as f:
    model = pickle.load(f)

future = model.make_future_dataframe(periods=7)
forecast = model.predict(future)

#
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(7)



import requests

# Endpoint /forecast
url = "http://127.0.0.1:8000/forecast"
payload = {
    "location": "Cádiz",
    "days": 5
}

response = requests.post(url, json=payload)
data = response.json()
data


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /forecast (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017C2413D010>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

In [ ]:
url = "http://127.0.0.1:8000/ask"
payload = {
    "question": "Temperatura máxima en Cádiz los últimos 7 días"
}

response = requests.post(url, json=payload)
data = response.json()
data


In [ ]:
import pandas as pd


df = pd.read_json("cadiz_final.json")
df = df.sort_values('fecha')

# Últimos 7 días tmax
df[['fecha','tmax']].tail(7)
